In [1]:
%load_ext autoreload

In [2]:
%autoreload 2
from dotenv import load_dotenv
import importlib
import sys
import os
import pickle
from pathlib import Path
load_dotenv()
# Add the parent directory of 'src' to the Python path
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

    
from src.config_loader import config, VoiceManager



In [2]:
def create_png_of_html(path_to_html, output_path=None, width=375, height=812, method='selenium'):
    """
    Renders an HTML file at smartphone screen size and saves it as a PNG.
    
    Args:
        path_to_html: Path to the HTML file to render
        output_path: Path where the PNG should be saved. If None, uses the HTML filename with .png extension
        width: Width of the smartphone screen in pixels (default: iPhone X width = 375)
        height: Height of the smartphone screen in pixels (default: iPhone X height = 812)
        method: Rendering method ('playwright', 'selenium', 'html2image', or 'auto')
        
    Returns:
        Path to the saved PNG file
    """
    import os
    import time
    
    if output_path is None:
        # Use the same filename but with .png extension
        output_path = os.path.splitext(path_to_html)[0] + '.png'
        
    if method == 'auto':
        # Try methods in order of preference
        methods = ['playwright', 'html2image', 'selenium']
        for m in methods:
            try:
                return create_png_of_html(path_to_html, output_path, width, height, method=m)
            except ImportError:
                continue
            except Exception as e:
                print(f"Method {m} failed: {str(e)}")
                continue
        raise ImportError("None of the required libraries are installed. Please install one of: playwright, html2image, or selenium.")
    
    elif method == 'playwright':
        try:
            from playwright.sync_api import sync_playwright
        except ImportError:
            raise ImportError("Playwright is not installed. Install with: pip install playwright && python -m playwright install")
        
        # Convert to absolute path for file:// URL
        abs_path = os.path.abspath(path_to_html)
        file_url = f"file://{abs_path}"
        
        with sync_playwright() as p:
            browser = p.chromium.launch()
            context = browser.new_context(
                viewport={"width": width, "height": height}
            )
            page = context.new_page()
            page.goto(file_url)
            
            # Wait for any resources to load
            page.wait_for_load_state("networkidle")
            
            # Give extra time for any JS to render
            time.sleep(0.5)
            
            # Take screenshot
            page.screenshot(path=output_path, full_page=True)
            
            # Clean up
            browser.close()
        
    elif method == 'selenium':
        try:
            from selenium import webdriver
            from selenium.webdriver.chrome.options import Options
        except ImportError:
            raise ImportError("Selenium is not installed. Install with: pip install selenium")
        
        # Setup Chrome in headless mode
        chrome_options = Options()
        chrome_options.add_argument("--headless")
        chrome_options.add_argument("--disable-gpu")
        chrome_options.add_argument(f"--window-size={width},{height}")
        
        driver = webdriver.Chrome(options=chrome_options)
        
        # Convert to absolute path for file:// URL
        abs_path = os.path.abspath(path_to_html)
        driver.get(f"file://{abs_path}")
        
        # Wait for any JS to render
        driver.implicitly_wait(2)
        time.sleep(0.5)
        
        # Take screenshot and save
        driver.save_screenshot(output_path)
        
        # Clean up
        driver.quit()
        
    elif method == 'html2image':
        try:
            from html2image import Html2Image
        except ImportError:
            raise ImportError("html2image is not installed. Install with: pip install html2image")
            
        # Create Html2Image instance
        output_dir = os.path.dirname(output_path) or '.'
        output_filename = os.path.basename(output_path)
        
        hti = Html2Image(output_path=output_dir, size=(width, height))
        
        # Generate PNG
        hti.screenshot(
            url=f"file://{os.path.abspath(path_to_html)}", 
            save_as=output_filename
        )
    else:
        raise ValueError(f"Unknown method: {method}. Use 'selenium', 'playwright', 'html2image', or 'auto'.")
        
    return output_path

In [6]:
create_png_of_html("../outputs/test_templates/listening_back.html", 
                   output_path="../outputs/test_templates/listening_back.png",
                   height=1100)

'../outputs/test_templates/listening_back.png'

In [11]:
story_phrases2 = [entry['phrase'] for entry in story_collection['story_teaching_music']]

In [9]:
story_phrases = [entry['phrase'] for entry in story_collection['story_midnight_garden_mystery']]

In [16]:
story_collection['story_midnight_garden_mystery']


[{'phrase': 'Did you hear? He hurt himself fixing the roof',
  'score': 9.0,
  'new_story_verbs': 1,
  'new_story_vocab': 0,
  'new_global_verbs': 1,
  'new_global_vocab': 2,
  'total_new_words': 3},
 {'phrase': 'Did you forget to include the data in the report?',
  'score': 9.0,
  'new_story_verbs': 1,
  'new_story_vocab': 0,
  'new_global_verbs': 1,
  'new_global_vocab': 2,
  'total_new_words': 3},
 {'phrase': "Don't the kids just love singing in the rain?",
  'score': 7.0,
  'new_story_verbs': 0,
  'new_story_vocab': 2,
  'new_global_verbs': 1,
  'new_global_vocab': 2,
  'total_new_words': 3},
 {'phrase': 'I want to produce a hit song',
  'score': 6.0,
  'new_story_verbs': 0,
  'new_story_vocab': 2,
  'new_global_verbs': 0,
  'new_global_vocab': 2,
  'total_new_words': 2},
 {'phrase': 'Even a low rate is better than nothing.',
  'score': 6.0,
  'new_story_verbs': 0,
  'new_story_vocab': 2,
  'new_global_verbs': 0,
  'new_global_vocab': 5,
  'total_new_words': 5},
 {'phrase': 'I love